In [ ]:
import os
import pandas as pd
import papermill as pm
import time
import numpy as np
from pathlib import Path
from optimization_helper import create_optimization_summary, create_optimization_animations, archive_optimization_run

# Configuration variables
NUM_ITERATIONS = 26

def run_optimization_loop():
    start_time = time.time()
    
    # Create necessary directories
    os.makedirs('visualization', exist_ok=True)
    os.makedirs('animations', exist_ok=True)
    os.makedirs('summary', exist_ok=True)
    
    # File paths
    optimization_file = "optimization.csv"
    suggestion_file = "suggestion.csv"
    
    # Load existing optimization data if available
    if os.path.exists(optimization_file):
        optimization_data = pd.read_csv(optimization_file)
        start_iteration = len(optimization_data)
    else:
        optimization_data = pd.DataFrame()
        start_iteration = 1
    
    print(f"Starting from iteration {start_iteration}")
    print(f"Current optimization data has {len(optimization_data)} samples")
    
    for iteration in range(start_iteration, start_iteration + NUM_ITERATIONS):
        print(f"\n{'='*50}")
        print(f"Iteration {iteration}")
        print(f"{'='*50}")
        iteration_start = time.time()
        
        try:
            # 1. Run Bayesian Optimization
            print("\nRunning Bayesian optimization...")
            pm.execute_notebook(
                "BO_Dev1.ipynb",  # This now contains our visualization code
                "temp_bo.ipynb",
                kernel_name="CaseStudy3"
            )
            
            # Check suggested parameters
            if os.path.exists(suggestion_file):
                suggestion = pd.read_csv(suggestion_file)
                print("\nSuggested parameters:")
                print(suggestion)
            else:
                raise FileNotFoundError("Suggestion file not found after BO step")
            
            # 2. Run Virtual Lab
            print("\nRunning virtual lab experiment...")
            pm.execute_notebook(
                "VirtualLabCS3.ipynb",
                "temp_vlab.ipynb",
                kernel_name="CaseStudy3",
                parameters={
                    "suggestion_file": suggestion_file,
                    "optimization_file": optimization_file
                }
            )
            
            # 3. Update optimization file with new results
            if os.path.exists(optimization_file):
                latest_results = pd.read_csv(optimization_file)
                if not latest_results.empty:
                    latest_row = latest_results.iloc[-1]
                    
                    # Create new result with all data
                    new_result = pd.DataFrame({
                        'T1Celsius': [suggestion['T1Celsius'].iloc[0]],
                        't1min': [suggestion['t1min'].iloc[0]],
                        'T2Celsius': [suggestion['T2Celsius'].iloc[0]],
                        't2min': [suggestion['t2min'].iloc[0]],
                        'EquivalentsReagent1': [suggestion['EquivalentsReagent1'].iloc[0]],
                        'EquivalentsBASE1': [suggestion['EquivalentsBASE1'].iloc[0]],
                        'ConcentrationMolar': [suggestion['ConcentrationMolar'].iloc[0]],
                        'Yield': [latest_row['Yield']],
                        'Impurity': [latest_row['Impurity']],
                        'ImpurityXRatio': [latest_row['ImpurityXRatio']]
                    })
                    
                    # Append to optimization file
                    optimization_data = pd.concat([optimization_data, new_result], ignore_index=True)
                    optimization_data.to_csv(optimization_file, index=False)
                    
                    print("\nNew result appended to optimization.csv:")
                    print(new_result)
                
            # 4. Create summary statistics and animations periodically
            if iteration % 5 == 0 or iteration == start_iteration + NUM_ITERATIONS - 1:
                print("\nCreating optimization summary...")
                create_optimization_summary(optimization_file)
                
                print("\nCreating progress animations...")
                create_optimization_animations(optimization_file)
            
            iteration_time = time.time() - iteration_start
            print(f"\nIteration {iteration} completed in {iteration_time:.2f} seconds")
            
        except Exception as e:
            print(f"Error in iteration {iteration}: {str(e)}")
            print("Detailed error information:")
            import traceback
            print(traceback.format_exc())
            break
    
    # Summary statistics
    total_time = time.time() - start_time
    print("\nOptimization Summary:")
    print(f"Total execution time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
    print(f"Initial samples: {start_iteration}")
    print(f"Added samples: {len(optimization_data) - start_iteration}")
    print(f"Total samples: {len(optimization_data)}")
    
    if not optimization_data.empty:
        print("\nOverall best results:")
        print(f"Best Yield: {optimization_data['Yield'].max():.2f}%")
        print(f"Lowest Impurity: {optimization_data['Impurity'].min():.2f}%")
        print(f"Best ImpurityXRatio: {optimization_data['ImpurityXRatio'].max():.2f}")
    
    # Archive the optimization run
    print("\nArchiving optimization run...")
    archive_path = archive_optimization_run(optimization_file)
    print(f"Run archived to {archive_path}")

if __name__ == "__main__":
    import warnings
    warnings.filterwarnings('ignore', category=Warning)
    run_optimization_loop()